In [5]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()


True

### Chroma

### 벡터 저장소 생성 (`from_document`)

 매개변수

- documents (List[Document]): 벡터 저장소에 추가할 문서 리스트
- embedding (Optional[Embeddings]): 임베딩 함수. 기본값은 None
- ids (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- collection_name (str): 생성할 컬렉션 이름.
- persist_directory (Optional[str]): 컬렉션을 저장할 디렉토리. 기본값은 None
- client_settings (Optional[chromadb.config.Settings]): Chroma 클라이언트 설정
- client (Optional[chromadb.Client]): Chroma 클라이언트 인스턴스
- collection_metadata (Optional[Dict]): 컬렉션 구성 정보. 기본값은 None

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")
loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)


(11, 6)

In [ ]:
db = Chroma.from_documents(
    documents=split_doc1,embedding=OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024),collection_name="my_db"
)

"""
Chroma.from_documents 메서드를 사용하여 메모리 내 임베딩 데이터베이스를 생성합니다.
persist_directory를 지정하지 않았으므로, 데이터베이스가 디스크에 저장되지 않고 메모리에만 존재합니다.
데이터베이스는 일시적입니다. 프로그램이 종료되면 데이터가 사라집니다.
"""

In [ ]:
DB_PATH = "./chroma_db"

persist_db = Chroma.from_documents(
    split_doc1,OpenAIEmbeddings(model="text-embedding-3-small"),persist_directory=DB_PATH,collection_name="my_db"
)

"""
Chroma.from_documents 메서드를 사용하여 데이터를 디스크에 저장하는 데이터베이스를 생성합니다.
persist_directory를 지정했기 때문에 데이터베이스가 ./chroma_db 경로에 저장됩니다.
디스크에 영구적으로 저장
"""

In [ ]:
# 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
    collection_name="my_db",
)
"""
기존에 저장된 데이터베이스(persist_directory에 저장된 데이터)를 다시 불러옵니다.
데이터베이스를 새로 생성하지 않고, 기존의 컬렉션("my_db")를 로드하여 작업을 수행합니다.
"""


In [11]:
# 저장된 데이터 확인
persist_db.get()


{'ids': ['03c35b18-113b-427b-a1eb-2360689d0c8d',
  '9a79e7ec-656a-4099-ab10-5fd0f8ca617a',
  'e615c977-68f9-47bb-a0f7-aeca2bc2af8f',
  'f81051d6-04fe-4265-b26b-55e0fade5574',
  'a78f249b-7a8f-493d-9801-cce38b894f3d',
  'c3f52b0e-f9f1-46f9-87ff-e075a37a88c0',
  '9d7ae24f-05cc-44d9-ab18-e7e5ab872956',
  '829863d3-a2f7-4dfc-9a29-c0054386d0bc',
  '39cfba3b-3310-4d96-b8c4-2a48b8fceb8c',
  '82687df6-bfe3-4ff5-90c2-98be02ca86ea',
  '23493179-bc49-4938-9ac2-7bea7f53d441'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 

In [12]:
# 디스크에서 문서를 로드합니다.
persist_db2 = Chroma(
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(),
    collection_name="my_db2",
)

# 저장된 데이터 확인
persist_db2.get()


{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소 생성 (`.from_txt`)

In [13]:
# 문자열 리스트로 생성
db2 = Chroma.from_texts(
    ["안녕하세요. 정말 반갑습니다.", "제 이름은 테디입니다."],
    embedding=OpenAIEmbeddings(),
)


In [14]:
db2.get()

{'ids': ['d1032ed5-090c-4621-a3ac-fe404408db14',
  '54bd0c93-0d70-4e26-b5fa-4f44e03f9143'],
 'embeddings': None,
 'documents': ['안녕하세요. 정말 반갑습니다.', '제 이름은 테디입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 유사도 검색

 `similarity_search` 메서드는 Chroma 데이터베이스에서 유사도 검색을 수행합니다. 이 메서드는 주어진 쿼리와 가장 유사한 문서들을 반환합니다.

#### 매개변수

- query (str): 검색할 쿼리 텍스트
- k (int, 선택적): 반환할 결과의 수. 기본값은 4입니다.
- filter (Dict[str, str], 선택적): 메타데이터로 필터링. 기본값은 None입니다.

In [18]:
db.similarity_search("TF IDF 에 대하여 알려줘",k=5)#k는 결과 갯수


[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키

In [20]:
# filter 사용
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "data/nlp-keywords.txt"}, k=2
)
# "source": "data/nlp-keywords.txt"인 metadata만 검색

[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키

### 벡터 저장소에 문서 추가

#### `add_documents` 메서드는 벡터 저장소에 문서를 추가하거나 업데이트합니다.

 매개변수

- documents (List[Document]): 벡터 저장소에 추가할 문서 리스트
- **kwargs: 추가 키워드 인자
- ids: 문서 ID 리스트 (제공 시 문서의 ID보다 우선함)

In [21]:
from langchain_core.documents import Document
db.add_documents(
    [
        Document(
            page_content="새로운 도큐먼트 추가",
            metadata = {'source':'mydata.txt'},
            id=1,
        )
    ]
)

['1']

In [23]:
db.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['새로운 도큐먼트 추가'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### `add_texts` 메서드는 텍스트를 임베딩하고 벡터 저장소에 추가합니다.

 매개변수

- texts (Iterable[str]): 벡터 저장소에 추가할 텍스트 리스트
- metadatas (Optional[List[dict]]): 메타데이터 리스트. 기본값은 None
- ids (Optional[List[str]]): 문서 ID 리스트. 기본값은 None

In [24]:
# 신규 데이터를 추가합니다. 이때 기존의 id=1 의 데이터는 덮어쓰게 됩니다.
db.add_texts(
    ["이전에 추가한 Document 를 덮어쓰겠습니다.", "덮어쓴 결과가 어떤가요?"],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["1", "2"],
)


['1', '2']

In [25]:
db.get(["1"])

{'ids': ['1'],
 'embeddings': None,
 'documents': ['이전에 추가한 Document 를 덮어쓰겠습니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소에서 문서 삭제

 `delete`메서드는 벡터 저장소에서 지정된 ID의 문서를 삭제합니다
 매개변수
- ids (Optional[List[str]]): 삭제할 문서의 ID 리스트. 기본값은 None

In [26]:
# id 1 삭제
db.delete(ids=["1"])


In [29]:
db.get("1")

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 초기화(reset_collection)
 `reset_collection`메서드는 벡터 저장소의 컬렉션을 초기화합니다.

In [30]:
# 컬렉션 초기화
db.reset_collection()


In [31]:
# 초기화 후 문서 조회
db.get()


{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소를 검색기(Retriever)로 변환
`as_retriever` 메서드는 벡터 저장소를 기반으로 VectorStoreRetriever를 생성합니다.
 매개변수
- **kwargs: 검색 함수에 전달할 키워드 인자
- search_type (Optional[str]): 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
- search_kwargs (Optional[Dict]): 검색 함수에 전달할 추가 인자
    - k: 반환할 문서 수 (기본값: 4)
    - score_threshold: 최소 유사도 임계값
    - fetch_k: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
    - lambda_mult: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5)
    - filter: 문서 메타데이터 필터링

In [32]:
# DB 생성
db = Chroma.from_documents(
    documents=split_doc1 + split_doc2,
    embedding=OpenAIEmbeddings(),
    collection_name="nlp",
)


In [33]:
retriever = db.as_retriever()
retriever.invoke("Word2Vec 에 대하여 알려줘")


[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 

- `similarity` : 쿼리 벡터와 문서 벡터 간 코사인 유사도를 기준으로 가장 유사한 문서들을 반환합니다
- `mmr` : 쿼리와 유사한 문서를 반환하면서도, 결과 문서 간 중복을 최소화하여 다양성을 증가시킵니다.
- `similarity_score_threshold` : 코사인 유사도가 특정 임계값을 초과하는 문서들만 반환합니다.

In [34]:
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)
retriever.invoke("Word2Vec 에 대하여 알려줘")


[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을

In [35]:
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10})
retriever.invoke("Word2Vec 에 대하여 알려줘")


[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝\n\nInstructGPT\n\n정의: Instruct

In [36]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)

retriever.invoke("Word2Vec 에 대하여 알려줘")


[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [38]:
#가장 유사한 단일 문서만 검색
retriever = db.as_retriever(search_kwargs={"k": 1})

retriever.invoke("Word2Vec 에 대하여 알려줘")


[Document(metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [39]:
#특정 메타데이터 필터 적용
retriever = db.as_retriever(
    search_kwargs={"filter": {"source": "data/finance-keywords.txt"}, "k": 2}
)
retriever.invoke("ESG 에 대하여 알려줘")


[Document(metadata={'source': 'data/finance-keywords.txt'}, page_content='정의: ESG는 기업의 환경, 사회, 지배구조 측면을 고려하는 투자 접근 방식입니다.\n예시: S&P 500 ESG 지수는 우수한 ESG 성과를 보이는 기업들로 구성된 지수입니다.\n연관키워드: 지속가능 투자, 기업의 사회적 책임, 윤리 경영\n\nStock Buyback\n\n정의: 자사주 매입은 기업이 자사의 주식을 시장에서 다시 사들이는 것을 말합니다.\n예시: 애플은 S&P 500 기업 중 가장 큰 규모의 자사주 매입 프로그램을 운영하고 있습니다.\n연관키워드: 주주 가치, 자본 관리, 주가 부양\n\nCyclical Stocks\n\n정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리'),
 Document(metadata={'source': 'data/finance-keywords.txt'}, page_content='정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.\n예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별 실적 전망을 발표했습니다.\n연관키워드: 투자 분석, 기업 가치평가, 시장 전망\n\nCorporate Governance\n\n정의: 기업 지배구조는 기업의 경영과 통제에 관한 시스템과 프로세스를 의미합니다.\n예시: S&P 500 기업들 중 이사회의 다양